In [ ]:
# !conda install -c conda-forge gdcm -y
!tar -xvf ../input/gdcm-conda-install/gdcm.tar
!conda install ../working/gdcm/conda-4.8.4-py37hc8dfbb8_2.tar.bz2
!conda install ../working/gdcm/gdcm-2.8.9-py37h71b2a6d_0.tar.bz2
!conda install ../working/gdcm/libjpeg-turbo-2.0.3-h516909a_1.tar.bz2

In [ ]:
from IPython.display import display_html
# def restartkernel() :
display_html("<script>Jupyter.notebook.kernel.restart()</script>")
# restartkernel()



In [ ]:
import numpy as np 
import pandas as pd 
from tqdm import tqdm
from glob import glob
import os
import matplotlib.pyplot as plt
import gdcm
import pydicom as dicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import pydicom
import cv2
from PIL import Image

In [ ]:
data_dir = "../input/siim-covid19-detection"
IM_SIZE = 512

In [ ]:
train_study_level = pd.read_csv(os.path.join(data_dir, "train_study_level.csv"))

In [ ]:
train_study_level.head()

# Dicom data

In [ ]:
train_df = train_study_level

In [ ]:
training_paths = []

for sid in tqdm(train_df['id']):
    path = glob(os.path.join(data_dir, 'train' , sid.split("_")[0] + "/*/*" ))[0]
    training_paths.append(path)

train_df['path'] = training_paths

In [ ]:
def dicom2array(path, voi_lut=True, fix_monochrome=True):
    dicom = pydicom.read_file(path)
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array

    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data


images = []
for filename in tqdm(train_df.path):
    im = dicom2array(filename)
    images.append(cv2.resize(im, (IM_SIZE, IM_SIZE)))

In [ ]:
os.makedirs("train_images", exist_ok=True)

In [ ]:
for i, idx in tqdm(enumerate(train_df.id)):
    Image.fromarray(images[i]).save(os.path.join("train_images", str(idx) + ".jpg"))

In [ ]:
!tar -czf train_images.tar.gz train_images
!du -h train_images.tar.gz

In [ ]:
from IPython.display import FileLink
FileLink(r'train_images.tar.gz')